## Antenna traffic single-step evaluation (Chronos)
We evaluate Chronos models on histo_trafic_....csv files by forecasting the last time step for each antenna sector using all prior measurements as context.
The helper scripts `dataprocessing.py` and `single_eval.py` handle preprocessing and RMSE scoring. The cells below run on device; edit `MODEL_IDS` to try more Chronos variants later and 'DATASET' to select the right dataset.jsonl to work on.


In [1]:
import pandas as pd

from scripts.notebook_helpers import (
    RESULTS_DIR,
    device,
    ensure_processed_data,
    evaluate_model,
)

DATASET = "instant"  # choose among: "orignal", "instant", "instant_short"
print("Using device:", device)
print("Using dataset:", DATASET)

ensure_processed_data(DATASET)



Using device: cpu
Using dataset: instant


PosixPath('data/processed_trafic_instant.jsonl')

In [2]:
MODEL_IDS = [
    "amazon/chronos-2",
    "amazon/chronos-bolt-tiny",
    "amazon/chronos-bolt-mini",
    "amazon/chronos-bolt-small",
    "amazon/chronos-bolt-base",
    "amazon/chronos-t5-tiny",
    "amazon/chronos-t5-mini",
    "amazon/chronos-t5-small",
    "amazon/chronos-t5-base",
    "amazon/chronos-t5-large",
]
NUM_SAMPLES = 32  # tweak for slower/fast CPU evaluations


In [3]:
results = [evaluate_model(mid, num_samples=NUM_SAMPLES, dataset=DATASET) for mid in MODEL_IDS]
summary_df = pd.DataFrame(
    {
        "model_id": [r["model_id"] for r in results],
        "rmse": [r["rmse"] for r in results],
        "num_series": [r["num_series"] for r in results],
        "num_samples": [r["num_samples"] for r in results],
    }
)
summary_df


Evaluating on 'instant': /Users/colinminini/Desktop/SCOC_ICE/LLM6G-Repository/.venv/bin/python scripts/single_eval.py --model-id amazon/chronos-2 --data-path data/processed_trafic_instant.jsonl --num-samples 32 --device cpu --output-path results/eval_amazon__chronos-2_instant.json


/Users/colinminini/Desktop/SCOC_ICE/LLM6G-Repository/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Model amazon/chronos-2 | RMSE=2.2920 across 86 sectors.
Saved detailed results to results/eval_amazon__chronos-2_instant.json
Evaluating on 'instant': /Users/colinminini/Desktop/SCOC_ICE/LLM6G-Repository/.venv/bin/python scripts/single_eval.py --model-id amazon/chronos-bolt-tiny --data-path data/processed_trafic_instant.jsonl --num-samples 32 --device cpu --output-path results/eval_amazon__chronos-bolt-tiny_instant.json
Model amazon/chronos-bolt-tiny | RMSE=2.1142 across 86 sectors.
Saved detailed results to results/eval_amazon__chronos-bolt-tiny_instant.json
Evaluating on 'instant': /Users/colinminini/Desktop/SCOC_ICE/LLM6G-Repository/.venv/bin/python scripts/single_eval.py --model-id amazon/chronos-bolt-mini --data-path data/processed_trafic_instant.jsonl --num-samples 32 --device cpu --output-path results/eval_amazon__chronos-bolt-mini_instant.json
Model amazon/chronos-bolt-mini | RMSE=2.1417 across 86 sectors.
Saved detailed results to results/eval_amazon__chronos-bolt-mini_instant

,model_id,rmse,num_series,num_samples
0,amazon/chronos-2,2.291993,86,32
1,amazon/chronos-bolt-tiny,2.114214,86,32
2,amazon/chronos-bolt-mini,2.141671,86,32
3,amazon/chronos-bolt-small,2.222061,86,32
4,amazon/chronos-bolt-base,2.192893,86,32
5,amazon/chronos-t5-tiny,2.490804,86,32
6,amazon/chronos-t5-mini,2.527757,86,32
7,amazon/chronos-t5-small,2.448102,86,32
8,amazon/chronos-t5-base,2.503635,86,32
9,amazon/chronos-t5-large,2.524886,86,32


In [4]:
PARAM_COUNTS = {
    "amazon/chronos-2": "120M",
    "amazon/chronos-bolt-tiny": "9M",
    "amazon/chronos-bolt-mini": "21M",
    "amazon/chronos-bolt-small": "48M",
    "amazon/chronos-bolt-base": "205M",
    "amazon/chronos-t5-tiny": "8M",
    "amazon/chronos-t5-mini": "20M",
    "amazon/chronos-t5-small": "46M",
    "amazon/chronos-t5-base": "200M",
    "amazon/chronos-t5-large": "710M",
}

order = MODEL_IDS
summary_ranked = summary_df.copy()
summary_ranked["parameters"] = summary_ranked["model_id"].map(PARAM_COUNTS)
summary_ranked["model_id"] = pd.Categorical(summary_ranked["model_id"], categories=order, ordered=True)
summary_ranked = summary_ranked.sort_values("model_id")[["model_id", "rmse", "parameters"]]

# Save the ranked summary to a CSV file
summary_ranked.to_csv(RESULTS_DIR / f"model_performance_summary_{DATASET}.csv", index=False)
summary_ranked

,model_id,rmse,parameters
0,amazon/chronos-2,2.291993,120M
1,amazon/chronos-bolt-tiny,2.114214,9M
2,amazon/chronos-bolt-mini,2.141671,21M
3,amazon/chronos-bolt-small,2.222061,48M
4,amazon/chronos-bolt-base,2.192893,205M
5,amazon/chronos-t5-tiny,2.490804,8M
6,amazon/chronos-t5-mini,2.527757,20M
7,amazon/chronos-t5-small,2.448102,46M
8,amazon/chronos-t5-base,2.503635,200M
9,amazon/chronos-t5-large,2.524886,710M
